In [29]:
import sys
import cohere
import os
import numpy as np
import math
import pickle

from pathlib import Path
# import openai_secret_manager
import openai
import re
import json
from dotenv import load_dotenv

result = os.path.dirname(os.path.realpath('__file__'))
relative_path = "../data/data.txt"
full_path = os.path.join(result, relative_path)

load_dotenv()
key = os.getenv('cohere_key')
co = cohere.Client(key)
openai.api_key =  os.getenv('openai_key')


In [30]:

file = open(full_path, "r")

texts = []
for line in file:
    texts.append(line.split(":")[1])

myRecipesFromData = texts

In [31]:
response = co.embed(
    texts=texts,
    model='small',
)
embeddings=response.embeddings
all_embeddings = np.array(embeddings)

relative_path = "../data/embeddings.npy"
full_path = os.path.join(result, relative_path)
np.save(full_path, all_embeddings)


89


In [ ]:
""" let's test here first """


response = co.embed(
    texts=["ginger, Lentils, tomatoes, onions, parsley, cilantro, vermicelli noodles, ginger, saffron, black pepper."],
    model='small',
)
embeddingForRecipeTest=response.embeddings

cosine_similarity_list = []
all_embeddings = np.load('embeddings.npy')
for i in range(len(all_embeddings)):
    # Calculate the dot product of the two arrays
    dot_product = np.dot(embeddingForRecipeTest, all_embeddings[i])

    # Calculate the norm of the two arrays
    norm_embeddingForRecipeTest = np.linalg.norm(embeddingForRecipeTest)
    norm_embedding = np.linalg.norm(all_embeddings[i])

    # Calculate the cosine similarity
    cosine_similarity = dot_product / (norm_embeddingForRecipeTest * norm_embedding)

    cosine_similarity_list.append((i, cosine_similarity))


# Sort the list by the variable in descending order
cosine_similarity_list.sort(key=lambda x: x[1], reverse=True)

# Select the top 3 by ID
top_3 = [x for x in cosine_similarity_list[:3]]






In [ ]:


# Authenticate with OpenAI API



# Define function to generate recipe suggestions
def generate_recipe_suggestions(ingredients, diet):
    if diet is not None:
        prompt = f"Generate a traditional Moroccan recipe using: {ingredients}. Be careful, The person has {diet}. For the response It should be a json object that contains, Name of the recipe, Ingredients, Instructions, Calories and the last one the date of today."
    else:
        prompt = f"Generate a traditional Moroccan recipe using: {ingredients}. For the response It should be a json object that contains, Name of the recipe, Ingredients, Instructions, Calories and the last one the date of today."
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    recipe = json.loads(json.dumps(response))["choices"][0]["text"]
    return recipe



diet = None

# Generate recipe suggestions and filter to only include traditional Moroccan recipes
recipe_suggestions = []
for recipe in top_3:
    generatedRecipe = generate_recipe_suggestions(myRecipesFromData[recipe[0]], diet)
    recipe_suggestions.append(json.loads(generatedRecipe))
    
print(recipe_suggestions)

[{'name': 'Moroccan Lentil Soup', 'ingredients': ['1 cup lentils', '1 cup chickpeas', '3 tomatoes', '1 onion', '1 celery stalk', '1/4 cup parsley', '1/4 cup cilantro', '1/4 cup lemon juice', '1/4 cup vermicelli noodles', '1 tablespoon olive oil', '1 teaspoon cumin', '1/2 teaspoon ginger', '1/4 teaspoon cinnamon', '1/4 teaspoon saffron', '1/2 teaspoon salt', '1/4 teaspoon black pepper'], 'instructions': ['1. In a large pot, combine the lentils, chickpeas, tomatoes, onion, celery, parsley, cilantro, lemon juice, vermicelli noodles, olive oil, cumin, ginger, cinnamon, saffron, salt, and black pepper.', '2. Add enough water to cover the ingredients.', '3. Bring to a boil over medium heat.', '4. Reduce the heat to low and simmer for 30 minutes.', '5. Serve hot.'], 'calories': ' Soup: 190 calories', 'date': "Today's date: March 3, 2020"}, {'recipe': {'name': 'Lamb Tagine with Chickpeas and Lentils', 'ingredients': ['1 lb lamb or beef', '1 onion', '3 cloves garlic', '1 celery stalk', '1 bunch